<a href="https://colab.research.google.com/github/Fu-kit/Final_Exam_12_November/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install pyinputplus (Required for Colab)
!pip install pyinputplus

# 2. Import Libraries
import pandas as pd
import pyinputplus as pyip